In [152]:
# Bayesian optimization 说白了就是通过不断的采样迭代,找出一个函数的全局最小/最大值
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pymodel.qmodel import *
import torch

from collections import Counter


np.random.seed(42)

In [153]:
# 定义模型和初始化参数
leaf = Leaf()

def fit_qmodel(X, w):  #@save
    """线性回归模型"""
        
    An_dict = Q_model_parameter(leaf = leaf,
                    Tair= X[0],co2 = X[1],RH = X[2],PAR = X[3],patm = X[4],
                    cost = w)

    return An_dict['An'][0]

# 定义模型和初始化参数
leaf = Leaf()
# initial_params = torch.tensor([0.11, 0.8, 0.001])

# 定义损失函数
def loss_fn(y_hat, y):
    return np.mean((y_hat - y) ** 2)

In [184]:
Q_model_parameter()

{'An': [11.323195680379873],
 'cp': [42.75],
 'Ci': [249.7500237342942],
 'Vc': [107.53221657542596],
 'E': [0.08546929283592213],
 'gs': [755.5950196883603]}

In [154]:
observation_df = pd.read_csv('./data/A_ci.csv')
observation_df = observation_df[observation_df['Photo'] > 0]

observation_df['RH'] = relative_humidity(observation_df['Tair'],observation_df['H2OS'],observation_df['Press'])
observation_df = observation_df[observation_df['RH'] <= 100]
observation_df = observation_df[observation_df['CO2S'] <= 450]
observation_df = observation_df[observation_df['CO2S'] >= 250]

Photo = observation_df['Photo'].tolist()
Tair = observation_df['Tair'].tolist()
CO2S = observation_df['CO2S'].tolist()
RH = (observation_df['RH']/100).tolist()
PARi = observation_df['PARi'].tolist()
Press = observation_df['Press'].tolist()

labels = torch.tensor(Photo)
features = torch.tensor([Tair,CO2S,RH,PARi,Press]).T

observation_df


,ID,Year,Season,Site,Plot,Tree/Individual,Branch,Rep,Photo,Ci,...,CO2R,CO2S,H2OR,H2OS,RH_R,RH_S,Flow,VpdA,Ci.Ca,RH
114,2015_dry_NOU-S1_103T1,2015,dry,NOU,S1,103,T,1,5.775690,235.512672,...,389.917572,382.220306,24.159431,26.104115,91.453278,98.815201,499.768036,1.033079,0.616170,96.759742
118,2015_dry_NOU-S3_71D1,2015,dry,NOU,S3,71,D,1,5.487268,233.422597,...,389.943420,383.191406,39.176125,39.611961,90.641075,91.649460,500.502045,0.481705,0.609154,88.464647
121,2015_dry_NOU-S3_71D1,2015,dry,NOU,S3,71,D,1,4.277795,263.075219,...,391.384796,386.087280,39.046032,39.466167,90.426643,91.399635,500.459381,0.489731,0.681388,88.235996
123,2015_dry_NOU-S3_71D2,2015,dry,NOU,S3,71,D,2,3.306961,193.536622,...,299.827393,295.724091,39.186848,39.637794,90.340691,91.380302,500.493286,0.535401,0.654450,88.202335
124,2015_dry_NOU-S3_71D2,2015,dry,NOU,S3,71,D,2,4.506136,247.989307,...,390.054138,384.470398,39.131313,39.584766,90.308800,91.355301,500.476440,0.531335,0.645015,88.182835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33527,2015_wet_PAR-B2_98T2,2015,wet,PAR,B2,98,T,2,1.612717,270.975238,...,389.254028,386.972046,16.312666,17.190662,38.355240,40.414734,499.736847,2.704257,0.700245,39.872412
33533,2015_wet_PAR-B2_98T2,2015,wet,PAR,B2,98,T,2,1.557725,271.958746,...,389.757812,387.547668,16.216885,17.078630,38.131813,40.116993,499.704865,2.725629,0.701743,39.582874
33615,2015_wet_PAR-T2_58T2,2015,wet,PAR,T2,58,T,2,5.158652,251.770500,...,389.887787,382.153198,-0.099078,3.917476,-0.224784,8.925460,499.739258,4.186858,0.658821,8.920464
33616,2015_wet_PAR-T2_58T2,2015,wet,PAR,T2,58,T,2,3.743112,197.520650,...,299.232697,293.566315,-0.109519,3.868572,-0.249500,8.829101,499.746521,4.176567,0.672831,8.825246


In [218]:
#  Bird Swarm Algorithm (BSA)
import numpy as np

X = features[0,]
Y = labels[0]

def objective_function(params):
    # 这个函数返回根据参数计算的模型结果和目标值之间的误差
    # 在这里,你需要将参数传递给你的模型,计算模型的结果,然后返回与目标值的误差
    # 假设你的模型是一个简单的函数,这里只是举个例子
    model_result = fit_qmodel(X,params)  # 假设模型是线性的
    target_value = Y
    error = abs(model_result - target_value)
    return error

def bird_swarm_algorithm(objective_function, bounds, n_iterations, n_birds=1000, 
                         c1=1.5, c2=1.5, max_error=0.01,dispersal_factor = 0.1):
    # 初始化鸟群的位置和速度
    birds_position = np.random.uniform(bounds[0], bounds[1], size=(n_birds, len(bounds[0])))
    birds_velocity = np.zeros_like(birds_position)

    # 计算每只鸟的适应度
    fitness = [objective_function(bird) for bird in birds_position]

    # 找到全局最优解
    global_best_position = birds_position[np.argmin(fitness)]
    global_best_error = objective_function(global_best_position)

    # 初始化当前误差最小的鸟的位置
    current_best_position = global_best_position.copy()
    best_position_list = []

    # 开始迭代
    for iteration in range(n_iterations):
        # 更新速度和位置
        for i in range(n_birds):
            
            # 保存可能的结果
            if fitness[i] < max_error:
                best_position_list.append(birds_position[i])
                
            r1, r2 = np.random.rand(2)
            birds_velocity[i] += c1 * r1 * (global_best_position - birds_position[i]) + c2 * r2 * (birds_position[i] - global_best_position)
            
            if abs(fitness[i] - global_best_error) < max_error/2:
                birds_velocity[i] *= 1 + np.random.uniform(-10, 10)

            # 如果当前鸟不是当前最小误差的鸟,就更新它的位置
            if not np.array_equal(birds_position[i], current_best_position):
                birds_position[i] += birds_velocity[i]

            # 限制位置在边界内
            birds_position[i] = np.clip(birds_position[i], bounds[0], bounds[1])

            # 计算适应度
            fitness[i] = objective_function(birds_position[i])

            # 更新全局最优解
            if fitness[i] < global_best_error:
                global_best_position = birds_position[i]
                global_best_error = fitness[i]
                
            
        # 更新当前误差最小的鸟的位置
        if global_best_error < objective_function(current_best_position):
            current_best_position = global_best_position.copy()

        # 打印当前迭代的结果
        if (iteration +1) % 5 == 0:
            print("Iteration:", iteration + 1, "Best error:", global_best_error)

        # 判断是否满足终止条件
        if global_best_error < max_error*0.1:
            break

    return current_best_position,best_position_list


In [221]:
# 设置参数的搜索范围
param_initial = [0.15,1.2, 0.001]
range_min = 0.001
range_max = 10
bounds = [np.array([param_initial[0] * range_min, param_initial[1] * range_min, param_initial[2] * range_min]), 
          np.array([param_initial[0] * range_max, param_initial[1] * range_max, param_initial[2] * range_max])]  # 假设参数范围是 (0.1, 10)

# 运行鸟群算法来寻找最优参数
param_list = bird_swarm_algorithm(objective_function, bounds,max_error=0.01, n_birds=10000, n_iterations=20)
param_list

Iteration: 5 Best error: tensor(0.0030)
Iteration: 10 Best error: tensor(0.0030)
Iteration: 15 Best error: tensor(0.0017)
Iteration: 20 Best error: tensor(0.0017)


(array([1.5583428e-01, 1.2000000e-03, 1.0000000e-06]),
 [array([1.96858737e-01, 1.20000000e-03, 1.00000000e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e+01, 1.0e-02]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e-04, 1.2e+01, 1.0e-02]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([3.0420705e-01, 1.2000000e-03, 1.0000000e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.57495402e-01, 1.20000000e-03, 1.00000000e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([1.03842386e+00, 1.20000000e+01, 1.00000000e-02]),
  array([1.5583428e-01, 1.2000000e-03, 1.0000000e-06]),
  array([1.5e+00, 1.2e-03, 1.0e-06]),
  array([0.08756184, 2.7294822 , 0.01      ]),
 

In [222]:
best_params = param_list[0]
print("Best parameters:", best_params,"X",X)
print("Model result:", fit_qmodel(X,w=[best_params[0],best_params[1],best_params[2]]))
print("Error:", objective_function(best_params))

Best parameters: [1.5583428e-01 1.2000000e-03 1.0000000e-06] X tensor([2.1998e+01, 3.8222e+02, 9.6760e-01, 1.2992e+03, 1.0051e+02])
Model result: tensor(5.7740)
Error: tensor(0.0017)


In [136]:
# 2. 在取直空间随机取值并放入黑箱模型中计算结果

a_range = np.linspace(0.01, 100, 1000) * 0.11
b_range = np.linspace(0.01, 100, 1000) * 0.8
c_range = np.linspace(0.01, 100, 1000) * 0.001


# random x values for sampling
num_samples = 30000
sample_a = np.random.choice(a_range, size=num_samples)
sample_b = np.random.choice(b_range, size=num_samples)
sample_c = np.random.choice(c_range, size=num_samples)

# output for each sampled x value

ob_i = 0
Y_hat = []
for i in range(0,num_samples):
    sample_y = fit_qmodel(features[ob_i,],w=[sample_a[i],sample_b[i],sample_c[i]])
    Y_hat.append(sample_y)

In [138]:
# 计算 Y_hat 与 labels[0] 之间的绝对差
errors = torch.abs(torch.tensor(Y_hat) - labels[ob_i])

# 获取前 20 个最小绝对误差值的索引
top_k_indices = torch.topk(errors, k=5, largest=False).indices
top_k_indices

tensor([ 2323, 22034,  7834, 29909, 14885])

In [139]:
for min_error_index in top_k_indices:
    best_para = [sample_a[min_error_index],sample_b[min_error_index],sample_c[min_error_index]]
    print(min_error_index,best_para,fit_qmodel(features[ob_i,],w=best_para),labels[ob_i],features[ob_i,])

tensor(2323) [0.08698698698698698, 2.8268268268268266, 6.006006006006006e-05] tensor(5.7347) tensor(5.7757) tensor([2.1998e+01, 3.8222e+02, 9.6760e-01, 1.2992e+03, 1.0051e+02])
tensor(22034) [0.13543543543543543, 1.0410410410410411, 0.0005905905905905906] tensor(5.6316) tensor(5.7757) tensor([2.1998e+01, 3.8222e+02, 9.6760e-01, 1.2992e+03, 1.0051e+02])
tensor(7834) [0.09909909909909909, 2.3703703703703702, 0.00019019019019019022] tensor(5.9904) tensor(5.7757) tensor([2.1998e+01, 3.8222e+02, 9.6760e-01, 1.2992e+03, 1.0051e+02])
tensor(29909) [0.14644644644644644, 0.5925925925925926, 0.0006706706706706706] tensor(5.4245) tensor(5.7757) tensor([2.1998e+01, 3.8222e+02, 9.6760e-01, 1.2992e+03, 1.0051e+02])
tensor(14885) [0.14974974974974972, 0.7527527527527528, 4.004004004004004e-05] tensor(5.2470) tensor(5.7757) tensor([2.1998e+01, 3.8222e+02, 9.6760e-01, 1.2992e+03, 1.0051e+02])
